In [2]:
import sys, os
sys.path.append('/Users/zaneselvans/code/catalyst/pudl')
sys.path.append('/Users/christinagosnell/code/pudl')
sys.path.append('/Users/Nannerz/Desktop/working/pudl/')
sys.path.append('/home/alana/Dropbox/catalyst/pudl/')
sys.path.append(os.path.abspath(os.path.join('..','..')))
from pudl import ferc1, pudl, models, models_ferc1, settings, constants, eia860, models_eia860
from pudl.eia860 import get_eia860_page
from pudl.eia860 import get_eia860_xlsx, get_eia860_column_map
# from pudl.eia923 import get_eia923_plants
from pudl.pudl import *
from pudl.constants import tab_map_eia860
import numpy as np
import pandas as pd
from pandas import DataFrame
import csv
import sqlalchemy as sa
from sqlalchemy import create_engine
from sqlalchemy.sql import select
from sqlalchemy import Integer, String, Numeric, Boolean, Float

In [3]:
pudl_engine  = pudl.db_connect_pudl()

In [174]:
#Pull in the generation_eia923 table
g_tbl = models.PUDLBase.metadata.tables['generation_eia923']
g_select = sa.sql.select([g_tbl,]) #creates a sql Select object
g = pd.read_sql(g_select, pudl_engine) #converts sql object to pandas dataframe

In [175]:
g9 = g.set_index(pd.DatetimeIndex(g['report_date']).year)
g9.drop('report_date',axis=1, inplace=True)
g9.reset_index(inplace=True)

In [176]:
g9_gb = g9.groupby(by=['plant_id','report_date','generator_id'])
g9_summed = g9_gb.agg({'net_generation_mwh':np.sum})

In [177]:
g9 = g9_summed.reset_index()

In [165]:
#Pull in the generator_eia860 table
g8_tbl = models.PUDLBase.metadata.tables['generators_eia860']
g8_select = sa.sql.select([g8_tbl,]) #creates a sql Select object
g8 = pd.read_sql(g8_select, pudl_engine) #converts sql object to pandas dataframe

In [166]:
g8 = g8[['plant_id','generator_id','nameplate_capacity_mw','summer_capacity_mw','winter_capacity_mw','year']]

In [167]:
g8['year'] = pd.to_datetime(g8['year'], format='%Y')
g8.rename(columns={'year': 'report_date'},inplace=True)
g8 = g8.set_index(pd.DatetimeIndex(g8['report_date']).year)

In [168]:
g8.drop('report_date',axis=1, inplace=True)
g8.reset_index(inplace=True)

In [179]:
capacity_factor = g9.merge(g8, on=['plant_id','generator_id','report_date'])

In [180]:
capacity_factor['capacity_factor'] =\
            capacity_factor['net_generation_mwh']/(capacity_factor['nameplate_capacity_mw']*8760)

In [181]:
capacity_factor

,plant_id,report_date,generator_id,net_generation_mwh,nameplate_capacity_mw,summer_capacity_mw,winter_capacity_mw,capacity_factor
0,3,2011,1,312130.000,153.1,138.0,138.0,0.232732
1,3,2011,2,191475.000,153.1,137.0,137.0,0.142769
2,3,2011,3,710069.000,272.0,249.0,249.0,0.298008
3,3,2011,4,1175685.000,403.7,362.0,362.0,0.332451
4,3,2011,5,2264413.000,788.8,726.5,726.5,0.327706
5,3,2011,A1ST,1397495.000,195.2,173.0,192.0,0.817271
6,3,2011,A2ST,1381781.000,195.2,173.0,192.0,0.808082
7,3,2012,1,152102.000,153.1,138.0,138.0,0.113411
8,3,2012,2,123410.000,153.1,137.0,137.0,0.092018
9,3,2012,3,496557.000,272.0,249.0,249.0,0.208399
